In [1]:
import pandas as pd
import numpy as np 
from py2neo import Node,Relationship,Graph, database,NodeMatcher 
import sys
import os

In [2]:
#primer archivo que crea nodos datasets, samples y la relacion entre ellos
a="C:/Users/espin/Bio-WS/Git_Scripts/metadata_PRJNA639275.csv" 
runs_metadata=pd.read_csv(a)
runs_metadata.set_axis([0,1,2,3,4], axis=1,inplace=True)

In [5]:
runs_metadata.head()

,0,1,2,3,4
0,SRR12007821,Convalescent,whole blood,male,SAMN15230292
1,SRR12007822,Convalescent,whole blood,male,SAMN15230292
2,SRR12007823,COVID-19,whole blood,female,SAMN15230291
3,SRR12007824,COVID-19,whole blood,female,SAMN15230291
4,SRR12007825,COVID-19,whole blood,male,SAMN15230290


In [77]:
graph=Graph()# or Graph('bolt://neo4j@localhost:7687')

In [82]:
runs_metadata[1][1], acc

('Convalescent', 'PRJNA639275')

In [87]:
 graph.run('MERGE(d:Database{name:$GEO}) MERGE(b:BioProyect{accession:$acc}) CREATE (d)<-[:IN]-(b)',GEO='GEO',acc=acc)


(No data)

In [72]:
i=4
SAMN=runs_metadata[4][i]
condition=runs_metadata[1][i]
acc, SAMN, condition

('PRJNA639275', 'SAMN15230290', 'COVID-19')

In [209]:
for i in range(0,len(runs_metadata[0])): 
    graph.run('MATCH(d:BioProyect{accession:$acc}) MERGE(s:Sample{SAMN:$SAMN}) MERGE(d)-[:HAVE{condition:$condition}]->(s)',acc=acc,SAMN=runs_metadata[4][i],condition=runs_metadata[1][i])


TAGER LOS DATASETS

In [210]:
lib='RNAseq'
ins='Illumina NovaSeq 6000'
subct='PBMC'
ct='Blood'
org='homo sapiens'
for i in runs_metadata[4].unique():
    graph.run("MATCH(a:Organism{name:$s}) match(b:Sample{SAMN:$code}) create(a)-[:TAGED]->(b)",code=i,s=org)
    graph.run("MATCH(a:Matrix{name:$s}) match(b:Sample{SAMN:$code}) create(a)-[:TAGED]->(b)",code=i,s=ct)
    graph.run("MATCH(a:SubMatrix{name:$s}) match(b:Sample{SAMN:$code}) create(a)-[:TAGED]->(b)",code=i,s=subct)
    graph.run("MATCH(a:Library{type:$s}) match(b:Sample{SAMN:$code}) create(a)-[:TAGED]->(b)",code=i,s=lib)
    graph.run("MATCH(a:Instrument{type:$s}) match(b:Sample{SAMN:$code}) create(a)-[:TAGED]->(b)",code=i,s=ins)

In [128]:
graph=Graph()# or Graph('bolt://neo4j@localhost:7687')

CREAR LAS CONCECCIONES ENTRE SAMPLE Y SUS GENES EXPRESADOS

In [6]:
# read.csv -> Conteo_gen para el sample 
h="C:/Users/espin/Bio-WS/PROYECTS/HUMAN-COVID/dataset_norm.csv" 
Conteo_gen=pd.read_csv(h)


In [114]:
Conteo_gen.head()

,Unnamed: 0,SRR12007821,SRR12007822,SRR12007823,SRR12007824,SRR12007825,SRR12007826,SRR12007827,SRR12007828,SRR12007829,...,SRR12007879,SRR12007880,SRR12007881,SRR12007882,SRR12007883,SRR12007884,SRR12007885,SRR12007886,SRR12007887,SRR12007888
0,ENSG00000000003.14,11,10,17,30,8,11,10,13,4,...,5,10,18,23,11,5,12,16,11,14
1,ENSG00000000005.5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ENSG00000000419.12,177,168,271,326,95,115,250,264,140,...,191,184,302,306,250,273,252,280,331,350
3,ENSG00000000457.13,229,215,211,226,150,159,175,182,128,...,216,198,346,360,245,229,273,255,306,313
4,ENSG00000000460.16,87,97,104,114,71,77,152,132,71,...,61,64,110,118,120,115,79,100,95,108


In [8]:
len(Conteo_gen)

35220

In [117]:
genes=Conteo_gen['Unnamed: 0']#si se corre dos veces va a tirar un error de que unnamed:0 no va a existir
genes = genes.str[:15]
Conteo_gen['Unnamed: 0']=genes
Conteo_gen.set_index('Unnamed: 0',inplace=True)

genes.head(),Conteo_gen[0].head()

(0    ENSG00000000003
 1    ENSG00000000005
 2    ENSG00000000419
 3    ENSG00000000457
 4    ENSG00000000460
 Name: Unnamed: 0, dtype: object,
                  SRR12007821  SRR12007822  SRR12007823  SRR12007824  \
 Unnamed: 0                                                            
 ENSG00000000003           11           10           17           30   
 ENSG00000000005            0            0            0            0   
 ENSG00000000419          177          168          271          326   
 ENSG00000000457          229          215          211          226   
 ENSG00000000460           87           97          104          114   
 
                  SRR12007825  SRR12007826  SRR12007827  SRR12007828  \
 Unnamed: 0                                                            
 ENSG00000000003            8           11           10           13   
 ENSG00000000005            0            0            0            0   
 ENSG00000000419           95          115          250       

In [200]:
samples=Conteo_gen.columns
samples=Conteo_gen.columns[0:]
#samples

In [212]:
import time

217.63985681533813

In [224]:
i=0
for s in runs_metadata[4].unique():
    segRUN=time.time()
    for ENSG in genes:
        av=(int(Conteo_gen[samples[0+i]][ENSG])+int(Conteo_gen[samples[(1+i)]][ENSG]))/2 #promedio valor normalizado expresin genes en las run de la sample
        if av>10:
            graph.run("MATCH(s:Sample{SAMN:$code}) MATCH(g:Gene{ENSEMBL:$ENSG}) CREATE (s)-[:EXPRESSED{count:$value}]->(g)",code=s,ENSG=ENSG,value=av)
    print('Completed! Sample:',s,samples[0+i],samples[(1+i)],round(time.time()-segRUN,2)/60,'min')
    i=i+2
       

Completed! Sample: SAMN15230292 SRR12007821 SRR12007822 3.1824999999999997 min
Completed! Sample: SAMN15230291 SRR12007823 SRR12007824 3.185166666666667 min
Completed! Sample: SAMN15230290 SRR12007825 SRR12007826 3.1768333333333336 min
Completed! Sample: SAMN15230289 SRR12007827 SRR12007828 3.263 min
Completed! Sample: SAMN15230288 SRR12007829 SRR12007830 3.2203333333333335 min
Completed! Sample: SAMN15230287 SRR12007831 SRR12007832 3.191333333333333 min
Completed! Sample: SAMN15230286 SRR12007833 SRR12007834 3.1151666666666666 min
Completed! Sample: SAMN15230285 SRR12007835 SRR12007836 3.0548333333333333 min
Completed! Sample: SAMN15230284 SRR12007837 SRR12007838 3.2871666666666663 min
Completed! Sample: SAMN15230283 SRR12007839 SRR12007840 3.2071666666666667 min
Completed! Sample: SAMN15230282 SRR12007841 SRR12007842 3.408 min
Completed! Sample: SAMN15230281 SRR12007843 SRR12007844 3.339 min
Completed! Sample: SAMN15230280 SRR12007845 SRR12007846 3.0541666666666667 min
Completed! Sam

In [207]:
runs_metadata.tail()

,0,1,2,3,4
63,SRR12007884,Healthy,whole blood,male,SAMN15230311
64,SRR12007885,Healthy,whole blood,female,SAMN15230310
65,SRR12007886,Healthy,whole blood,female,SAMN15230310
66,SRR12007887,Healthy,whole blood,male,SAMN15230306
67,SRR12007888,Healthy,whole blood,male,SAMN15230306
